##Importing libraries and packages

In [ ]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle # mixing up or currently ordered data that might lead our network astray in training.
from tqdm import tqdm      # Faster iteration
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras
TRAIN_DIR = 'drive/MyDrive/Colab Notebooks/OurData/train'
TEST_DIR = 'drive/MyDrive/Colab Notebooks/OurData/test'
IMG_SIZE = 100
LR = 1e-3

MODEL_NAME = 'ripeVSunripe-{}-{}.model'.format(LR, '2conv-basic') 

##Converting labels to 1 for ripe & 0 for unripe

In [ ]:
def label_img(img):
    word_label = img.split(' ')[0]
    # conversion to one-hot array [Ripe,Unripe]
    #                            [very Ripe, almost Unripe]
    if word_label == 'Ripe': return 1
    #                             [almost Ripe, very Unripe]
    elif word_label == 'Unripe': return 0

##Converting train images to feature and label 

In [ ]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [ ]:
train_data = create_train_data()

##Make matplot understand cv2 for trainelements

In [ ]:
Trainf=[]
TrainfM=[]
trainl=[]
for i in range(len(train_data)):
  Trainf.append(train_data[i][0])
  TrainfM.append(cv2.cvtColor(Trainf[i], cv2.COLOR_BGR2RGB))
  trainl.append(train_data[i][1])

##Converting test images to feature and label

In [ ]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img), img_num])
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data
    

In [ ]:
test_data= process_test_data()
#test_data

##Make matplot understand cv2 for testelements

In [ ]:
Testf=[]
TestfM=[]
for i in range(len(test_data)):
  Testf.append(test_data[i][0])
  TestfM.append(cv2.cvtColor(Testf[i], cv2.COLOR_BGR2RGB))

##convert 1 => Ripe & 0 => Unripe

In [ ]:
kind = {'Ripe':1 ,'Unripe':0}

def getkind(n) : 
    for x , y in kind.items() : 
        if n == y : 
            return x

##Plot train data

In [ ]:
#plt.figure(figsize=(20,20))
#for n , i in enumerate(list(np.random.randint(0,len(train_data),12))) : 
    #plt.subplot(6,6,n+1)
    #plt.imshow(TrainfM[i])   
    #plt.axis('off')
    #plt.title(getkind(train_data[i][1]))

##Plot test data

In [ ]:
#plt.figure(figsize=(20,20))
#for n , i in enumerate(list(np.random.randint(0,len(test_data),12))) : 
    #plt.subplot(6,6,n+1)
    #plt.imshow(TestfM[i])   
    #plt.axis('off')
    #plt.title(getkind(test_data[i][1]))

##PREPARE DATA

In [ ]:
X_train = TrainfM[:-250]
X_test = TrainfM[-250:]
X_pred_array = TestfM
y_train = trainl[:-250]
y_test = trainl[-250:]
#--------------------------------
X_train= np.array(X_train)
X_test=np.array(X_test)
X_pred_array=np.array(X_pred_array)
y_train=np.array(y_train)
y_test=np.array(y_test)
#---------------------------------
print(f'X_train shape  is {X_train.shape}')
print(f'X_test shape  is {X_test.shape}')
print(f'X_pred shape  is {X_pred_array.shape}')
print(f'y_train shape  is {y_train.shape}')
print(f'y_test shape  is {y_test.shape}')

##((-$ OUR MODEL $-))


In [ ]:
KerasModel = keras.models.Sequential([
        keras.layers.Conv2D(200,kernel_size=(3,3),activation='relu',input_shape=(IMG_SIZE,IMG_SIZE,3)),
        keras.layers.Conv2D(150,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Conv2D(120,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(80,kernel_size=(3,3),activation='relu'),    
        keras.layers.Conv2D(50,kernel_size=(3,3),activation='relu'),
        keras.layers.MaxPool2D(4,4),
        keras.layers.Flatten() ,    
        keras.layers.Dense(120,activation='relu') ,    
        keras.layers.Dense(100,activation='relu') ,    
        keras.layers.Dense(50,activation='relu') ,        
        keras.layers.Dropout(rate=0.5) ,            
        keras.layers.Dense(6,activation='softmax') ,    
        ])

In [ ]:
KerasModel.compile(optimizer ='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
print('Model Details are : ')
print(KerasModel.summary())

In [ ]:
epochs=20
ThisModel = KerasModel.fit(X_train,y_train, epochs=epochs,batch_size=64,verbose=1)

In [ ]:
ModelLoss, ModelAccuracy = KerasModel.evaluate(X_test, y_test)

print('Test Loss is {}'.format(ModelLoss))
print('Test Accuracy is {}'.format(ModelAccuracy ))

In [ ]:
y_pred = KerasModel.predict(X_test)

print('Prediction Shape is {}'.format(y_pred.shape))

In [ ]:
y_result = KerasModel.predict(X_pred_array)

print('Prediction Shape is {}'.format(y_result.shape))

In [ ]:
plt.figure(figsize=(20,20))
for n , i in enumerate(list(np.random.randint(0,len(X_pred_array),36))) : 
    plt.subplot(6,6,n+1)
    plt.imshow(X_pred_array[i])    
    plt.axis('off')
    plt.title(getkind(np.argmax(y_result[i])))